# 설정

In [ ]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 그래프 출력
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 데이터 준비

## 데이터 다운로드

In [ ]:
import os
import tarfile
import urllib.request

# csv를 압축한 tgz 파일 다운로드
# 압축 풀기 등 다운로드 작업을 자동화 -> 데이터 추출하는 함수(fetch_housing_data)

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

# fetch_housing_data 실행
# 현재 작업 공간에 datasets/housing 디렉터리를 생성
# housing.tgz 파일을 내려받음
# 압축을 푼 후 housing.csv 파일로 만듦


In [ ]:
fetch_housing_data()

In [ ]:
import pandas as pd

# housing.csv 불러오기

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

## 데이터 분포 출력

In [ ]:
# 10개의 특성을 가짐
sw181133 = load_housing_data()
sw181133.head()

In [ ]:
# info() 메서드
# 데이터에 대한 간략한 설명, 전체 행 수, 각 특성의 데이터 타입, null이 아닌 값의 개수
sw181133.info()
# 대부분이 수치형 데이터
# ocean_proximity 필드가 유일한 범주형 데이터 -> 확인 필요

In [ ]:
# 카테고리 종류와 카테고리당 구역(행)의 수
sw181133["ocean_proximity"].value_counts()

In [ ]:
# 숫자형 특성의 요약 정보
sw181133.describe()

In [ ]:
# 모든 숫자형 특성에 대한 히스토그램
# 주어진 값의 범위(수평축), 샘플 수(수직축)

%matplotlib inline
import matplotlib.pyplot as plt
sw181133.hist(bins=50, figsize=(20,15))
save_fig("attribute_histogram_plots")
plt.show()

# 중간 소득 특성이 US 달러로 표현되어 있지 않음
# median_house_value의 상한값(최대값)이 있어서 예측이 이 값을 넘지 않는다.
# 해결 1. 한계값 밖의 정확한 값 알아내기
# 해결 2. 훈련세트/테스트세트에서 제거
# 전체적으로 특성들의 스케일이 다름 -> 제대로 특성이 반영이 안됨
# 특성이 왼쪽보다 오른쪽으로 멀리 뻗어 있음 -> 분포 조정 필요

## 테스트 세트 생성

데이터 스누핑 편향(data snooping)
* 테스트 세트로 일반화 오차를 추정 -> 낙관적인 추정이 됨
* 특정 머신러닝 모델을 선택하게 됨으로써 시스템 론칭 시 기대한 성능이 안 나옴

In [ ]:
# 실행 값이 동일하도록 random seed 값 고정
np.random.seed(42)

In [ ]:
# 무작위로 어떤 샘플을 선택해서 데이터 세트의 20% 분할

import numpy as np

def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train_set, test_set = split_train_test(sw181133, 0.2)
len(train_set)

In [ ]:
len(test_set)

In [ ]:
# 위 코드를 다시 실행하면 다른 테스트 세트가 생성됨
# 여러 번 계속하게 되면 전체 데이터 세트를 보는 것과 마찬가지이므로 다른 방법이 필요함
# 해결 1. 처음 실행에서 테스트 세트를 저장하고 다음번 실행에 불러오기
# 해결 2. 항상 같은 난수 인덱스가 생성되도록 np.random.permutation() 호출 전에 난수 발생기의 초깃값을 지정
# 두 해법 모두 다음번에 업데이트된 데이터 세트를 사용하려면 문제가 생김

# 해결책 -> 샘플의 식별자를 사용하여 테스트 세트에 보낼지 말지 정하는 것
# ex) 각 샘플마다 식별자의 해시값을 계산하여 해시 최댓값의 20%보다 작거나 같은 샘플만 테스트 세트로 보냄
# 여러 번 반복 실행되어 데이터 세트가 갱신되더라도 테스트 세트가 동일하게 유지됨
# 새로운 테스트 세트는 샘플의 20%를 갖게 되나, 이전에 훈련 세트에 있던 샘플은 포함시키지 않음

from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]
    

In [ ]:
import hashlib

def test_set_check(identifier, test_ratio, hash=hashlib.md5):
    return hash(np.int64(identifier)).digest()[-1] < 256 * test_ratio

In [ ]:
def test_set_check(identifier, test_ratio, hash=hashlib.md5):
    return bytearray(hash(np.int64(identifier)).digest())[-1] < 256 * test_ratio

In [ ]:
# 주택 데이터 세트에는 식별자 컬럼이 없음 -> 행의 인덱스를 id로 사용
housing_with_id = sw181133.reset_index()  # index 열이 추가된 데이터프레임을 반환
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

In [ ]:
# 행의 인덱스를 고유 식별자로 사용할 때 새 데이터는 데이터 세트의 끝에 추가되어야 함
# 고유 식별자를 만드는 데 안전한 특성을 사용해야 함, ex) 위도/경도 
housing_with_id["id"] = sw181133["longitude"] * 1000 + sw181133["latitude"]
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "id")

In [ ]:
test_set.head

In [ ]:
# train_test_split()
# 무작위 샘플링에 의한 훈련/테스트 분할
# random_state : 난수 초기값 지정
# 행의 개수가 같은 여러 개의 데이터 세트를 넘겨서 같은 인덱스 기반으로 나눌 수 있음
# 데이터프레임이 레이블을 따라 여러 개로 나뉘어 있을 때 유용

from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(sw181133, test_size=0.2, random_state=42)

In [ ]:
test_set.head()

In [ ]:
# 중간소득의 카테고리 빈도에 따라 훈련세트/테스트세트 조정
sw181133["median_income"].hist()

In [ ]:
# pd.cut으로 5개 소득 카테고리 지정
# 카테고리 레이블 1은 0에서 1.5까지 범위를 의미
# 카테고리 레이블 2는 1.5에서 3까지 범위를 의미

sw181133["income_cat"] = pd.cut(sw181133["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])


In [ ]:
sw181133["income_cat"].value_counts()

In [ ]:
sw181133["income_cat"].hist()

In [ ]:
# 소득 카테고리 기반 계층 샘플링
# 계층 샘플링을 하는 이유 -> 테스트 세트가 전체 데이터를 대표하도록 각 계층에서 올바른 수의 샘플을 추출

from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(sw181133, sw181133["income_cat"]):
    strat_train_set = sw181133.loc[train_index]
    strat_test_set = sw181133.loc[test_index]
    

In [ ]:
# 테스트데이터에서 소득 카테고리별 빈도수
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
# 전체데이터 카테고리별 빈도수
# 위와 거의 수치가 비슷함을 알 수 있음
sw181133["income_cat"].value_counts() / len(sw181133)

In [ ]:
# 전체 데이터 세트에 있는 소득 카테고리의 비율을 측정
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(sw181133, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall": income_cat_proportions(sw181133),
    "Stratified": income_cat_proportions(strat_test_set),
    "Random": income_cat_proportions(test_set),
}).sort_index()
compare_props["Rand. %error"] = 100 * compare_props["Random"] / compare_props["Overall"] - 100
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

In [ ]:
# 계층 샘플링을 사용해 만든 테스트 세트가 전체 데이터 세트에 있는 소득 카테고리의 비율과 거의 같음
# 전체/계층 샘플링/무작위 샘플링/무작위 샘플링 오류율/계층 샘플링 오류율
compare_props

In [ ]:
# income_cat 특성 삭제
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# 데이터 상관관계 출력

In [ ]:
# 표준상관계수(피어슨 상관계수)
corr_matrix = sw181133.corr()

In [ ]:
# 중간주택 가격과 다른 특성 사이의 상관관계 크기
# 관계 범위 -1 1
# 1에 가까우면 양의 상관관계, 한쪽이 올라가면 같이 증가
# 계수가 0에 가까우면 선형적인 상관관계가 없음
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# 두 값 사이를 산점도로 파악
# 대각선은 자신에 대한 히스토그램
# 숫자형 특성 사이의 산점도를 그려줌
# 중간 주택 가격을 예측하는데 가장 유용한 것은 중간 소득임을 알 수 있음
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(sw181133[attributes], figsize=(12, 8))
save_fig("scatter_matrix_plot")

In [ ]:
# 상관관계가 비교적 강함
# 가격 제한값이 설정돼있음
sw181133.plot(kind="scatter", x="median_income", y="median_house_value",
             alpha=0.1)
plt.axis([0, 16, 0, 550000])
save_fig("income_vs_house_value_scatterplot")

# 새로운 feature 추가

In [ ]:
# 가구 수 대비 방의 개수
# 전체방 수 대비 침실 수
# 가구 수 대비 인구 수
sw181133["rooms_per_household"] = sw181133["total_rooms"]/sw181133["households"]
sw181133["bedrooms_per_room"] = sw181133["total_bedrooms"]/sw181133["total_rooms"]
sw181133["population_per_household"]=sw181133["population"]/sw181133["households"]

In [ ]:
# 기존의 것보다는 특성 조합한 게 상관관계가 더 높게 나옴
# 전체방 수 대비 침실 수가 낮으면 집값이 더 비싸다 -> 음의 상관관계
corr_matrix = sw181133.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

# 파이프라인 설계

**함수를 만들어 자동화해야 하는 이유**
* 어떤 데이터 세트에 대해서도 데이터 변환을 손쉽게 반복할 수 있음
* 향후 프로젝트에 사용할 수 있는 변환 라이브러리를 점진적으로 구축하게 됨
* 실제 시스템에서 알고리즘에 새 데이터를 주입하기 전에 변환시키는데 이 함수를 사용할 수 있음
* 여러 가지 데이터 변환을 쉽게 시도해볼 수 있고 어떤 조합이 가장 좋은지 확인하는 데 편리함

In [ ]:
sw181133 = strat_train_set.drop("median_house_value", axis=1) # 훈련 세트를 위해 레이블 삭제
housing_labels = strat_train_set["median_house_value"].copy()

## 데이터 정제

```
housing.dropna(subset=["total_bedrooms"])    # 옵션 1
housing.drop("total_bedrooms", axis=1)       # 옵션 2
median = housing["total_bedrooms"].median()  # 옵션 3
housing["total_bedrooms"].fillna(median, inplace=True)
```
**값이 없는 경우**
* 해당 구역을 제거
* 전체 특성을 삭제
* 어떤 값으로 채움 (0, 평균, 중간값)

In [ ]:
# 적어도 하나의 열이 비어있는 행을 고름
sample_incomplete_rows = sw181133[sw181133.isnull().any(axis=1)].head()
sample_incomplete_rows

In [ ]:
# na not available
sample_incomplete_rows.dropna(subset=["total_bedrooms"])    # 옵션 1

In [ ]:
# 컬럼(열, 특성) 삭제
sample_incomplete_rows.drop("total_bedrooms", axis=1)       # 옵션 2

In [ ]:
# median 값을 구한 후, fillna로 중간값 채우기
median = sw181133["total_bedrooms"].median()
sample_incomplete_rows["total_bedrooms"].fillna(median, inplace=True) # 옵션 3

In [ ]:
sample_incomplete_rows

In [ ]:
# KNNImputer
# median 값으로 채움
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

In [ ]:
# 중간값이 수치형 특성에서만 사용될 수 있기 때문에 텍스트 특성을 삭제
housing_num = sw181133.drop("ocean_proximity", axis=1)
# 다른 방법: housing_num = housing.select_dtypes(include=[np.number])

In [ ]:
imputer.fit(housing_num)

In [ ]:
# 각 컬럼별 median 값
imputer.statistics_

In [ ]:
# 수동으로 계산한 값과 같음을 확인할 수 있음
housing_num.median().values

In [ ]:
# 넘파이 배열로 나옴
X = imputer.transform(housing_num)

In [ ]:
# 판다스 데이터프레임으로 변환
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index=housing_num.index)

In [ ]:
housing_tr.loc[sample_incomplete_rows.index.values]

In [ ]:
imputer.strategy

In [ ]:
housing_tr.head()

## 텍스트와 범주형 특성 정제

범주형 입력 특성인 `ocean_proximity`을 전처리

In [ ]:
housing_cat = sw181133[["ocean_proximity"]]
housing_cat.head(10)

In [ ]:
# text를 숫자로 변환
# OrdinalEncoder : 순서가 있고 값으로 비교 가능
# bad average good excellent 
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
ordinal_encoder.categories_

## 원핫인코딩 적용

In [ ]:
# OneHotEncoder : 한 특성만 1이고 나머지는 0, 기본적으로 희소행렬로 반환
# sparse 행렬로 저장 : 0이 아닌 위치만 저장 
# 희소행렬로 만들면 메모리 사이즈가 줄어 쉽게 불러오기 가능
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
# toarray()를 사용해 밀집 행렬로 반환
housing_cat_1hot.toarray()

In [ ]:
# sparse를 False로 설정해줌
cat_encoder = OneHotEncoder(sparse=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
# 카테고리 리스트 보기
cat_encoder.categories_

## 나만의 변환기

In [ ]:
# 조합 특성을 추가하는 변환기
from sklearn.base import BaseEstimator, TransformerMixin

# 열 인덱스
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # *args 또는 **kargs 없음
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # 아무것도 하지 않습니다
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(sw181133.to_numpy())

In [ ]:
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(sw181133.columns)+["rooms_per_household", "population_per_household"],
    index=sw181133.index)
housing_extra_attribs.head()

## 변환 파이프라인 & 스케일링

수치형 특성을 전처리하기 위해 파이프라인

In [ ]:
# CombinedAttributesAdder : 특성을 붙임
# 연속된 변환을 순서대로 처리할 수 있도록 도와주는 Pipeline 클래스
# 숫자 특성을 처리하는 간단한 파이프라인

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
# 하나의 변환기로 각 열마다 다른 변환 적용
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

# 전체 주택 데이터를 받아 각 열에 적절한 변환을 적용하는 전처리 파이프라인 생성
# 수치형 특성은 num_pipeline을 사용해 변환되고 범주형 특성은 OneHotEncoder를 사용해 변환됨
# ColumnTransformer이 밀집도(0이 아닌 원소의 비율)가 임계값보다 낮으면 희소행렬로 반환
# num_pipeline은 밀집행렬, OneHotEncoder는 희소행렬 반환
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(sw181133)

# 모델 선택과 훈련

## 선형회귀 모델 적용

In [ ]:
# 선형 회귀 모델 생성
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
# 훈련 샘플 몇 개를 사용해 전체 파이프라인을 적용
some_data = sw181133.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("예측:", lin_reg.predict(some_data_prepared))

In [ ]:
# 실제값과 비교
print("레이블:", list(some_labels))

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

# 예측오차 68627 -> 과소적합
# 문제점 : 특성들이 좋은 정보를 제공하지 못했거나 모델이 강력하지 못함
# 해결법 : 강력한 모델 선택, 훈련 알고리즘에 더 좋은 특성 주입, 모델의 규제 감소
# 이 선형 모델에서는 모델의 규제를 사용하지 않았으므로 마지막 해결법은 제외
# 또다른 해결법 : 특성 추가(인구의 로그스케일), 다른 모델 적용

## 결정트리 모델 적용

In [ ]:
# 결정트리
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
# 오차가 0
# 훈련데이터 과적합
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

## 교차 검증

In [ ]:
# 결정트리 교차검증
# 교차검증에서 scoring을 효용함수(클수록 좋은 값)로 계산
# K겹 교차검증, 훈련 세트를 폴드라 불리는 10개의 서브셋으로 무작위로 분할
# 매번 다른 폴드를 선택해 평가에 사용하고 나머지 9개 폴드는 훈련에 사용
# 결정 트리 모델을 10번 훈련하고 평가함
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("점수:", scores)
    print("평균:", scores.mean())
    print("표준 편차:", scores.std())

display_scores(tree_rmse_scores)

# 각 훈련의 평균과 표준편차를 통해 모델의 성능과 추정이 얼마나 정확한지 추정

In [ ]:
# 선형 교차검증
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

In [ ]:
# 결정트리 모델이 과대적합되어 선형 회귀 모델보다 성능이 나쁨

## 랜덤포레스트 모델 적용

In [ ]:
# 100개의 트리 생성
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

In [ ]:
# 랜덤포레스트 RMSE 계산
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
# 앙상블 학습 : 여러 다른 모델을 모아서 하나의 모델을 만드는 것, 머신러닝 알고리즘의 성능을 극대화
# 훈련 세트에 대한 점수가 검증 세트에 대한 점수보다 낮음 -> 과대적합임을 알 수 있음
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

# 모델 세부 튜닝

## 그리드서치 적용

In [ ]:
# GridSearchCV는 비교적 적은 수의 조합을 탐구할 때 좋음
# 모델을 세부적으로 튜닝하는 법 -> 만족할 만한 하이퍼파라미터의 조합을 찾을 때까지 파라미터 값 조정
# GridSearchCV를 이용해 탐색하고자 하는 하이퍼파라미터와 시도해볼 값을 지정
# 모든 하이퍼파라미터 조합에 대해 교차 검증을 사용해 평가
# RandomForestRegressor에 대한 최적의 하이퍼파라미터 조합을 탐색
from sklearn.model_selection import GridSearchCV

param_grid = [
    # 12(=3×4)개의 하이퍼파라미터 조합을 시도
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # bootstrap은 False로 하고 6(=2×3)개의 조합을 시도
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# 다섯 개의 폴드로 훈련하면 총 (12+6)*5=90번의 훈련이 일어남
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
# 최상의 조합 보기
grid_search.best_params_

In [ ]:
# 최적치 추정기
grid_search.best_estimator_

In [ ]:
# 그리드서치를 통해 테스트한 하이퍼파라미터 조합 평가 점수
# n_estimators가 30, max_features가 8일 때 최적의 솔루션
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
pd.DataFrame(grid_search.cv_results_)

## 랜덤서치 적용

In [ ]:
# RandomizedSearchCV는 하이퍼파라미터의 탐색 공간이 커지면 사용하는 편
# 가능한 모든 조합을 시도하는 대신 각 반복마다 하이퍼파라미터에 임의의 숫자 대입, 지정한 횟수만큼 평가
# 설정값이 연속형일 경우 추천
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(housing_prepared, housing_labels)

In [ ]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

## 최상의 모델과 오차 분석

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
#cat_encoder = cat_pipeline.named_steps["cat_encoder"] # 예전 방식
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

## 시스템 평가 / 최종 결과 출력

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

# 훈련세트가 아니므로 fit_transform이 아닌 transform을 써야 함
X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

# 테스트 데이터에 적용한 값이 더 낮은 경우가 일반적
# 테스트 데이터를 위한 하이퍼 파라미터 조정은 바람직하지 않음

In [ ]:
final_rmse